## Задание к теме «Гибридные рекомендательные системы»

Что делать?

Датасет ml-latest
1. Вспомнить подходы, которые мы разбирали
2. Выбрать понравившийся подход к гибридным системам
3. Написать свою

In [1]:
import pandas as pd
from surprise import KNNBaseline
from surprise import Dataset
from surprise import Reader
from surprise import accuracy

In [2]:
links = pd.read_csv('links.csv')
movies = pd.read_csv('movies.csv')
ratings = pd.read_csv('ratings.csv')
tags = pd.read_csv('tags.csv')

In [3]:
movies_with_ratings = movies.join(ratings.set_index('movieId'), on='movieId').reset_index(drop=True)
movies_with_ratings.dropna(inplace=True)

In [4]:
dataset = pd.DataFrame({
    'uid': movies_with_ratings.userId,
    'iid': movies_with_ratings.title,
    'rating': movies_with_ratings.rating
})

In [5]:
reader = Reader(rating_scale=(dataset.rating.min(), dataset.rating.max()))
data = Dataset.load_from_df(dataset, reader)

In [6]:
from surprise.model_selection import train_test_split
trainset, testset = train_test_split(data, test_size=0.15)

In [7]:
algo1 = KNNBaseline(k=40, min_k=20, sim_options={'name': 'pearson_baseline', 'user_based': True})
algo1.fit(trainset)

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.


In [8]:
algo1.predict(uid=15, iid='Addams Family Values (1993)').est

3.0021260181056277

In [9]:
test_pred = algo1.test(testset)
accuracy.rmse(test_pred, verbose=True)

RMSE: 0.8590


0.8590042967011458

In [10]:
from surprise import CoClustering
algo2 = CoClustering()
algo2.fit(trainset)

C:\Users\BEU_RU1\AppData\Local\Temp/ipykernel_15112/4043758920.py:3: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  algo2.fit(trainset)


In [11]:
algo2.predict(uid=15, iid='Addams Family Values (1993)').est

2.580037730902554

In [12]:
test_pred = algo2.test(testset)
accuracy.rmse(test_pred, verbose=True)

RMSE: 0.9443


0.9442637511216485

### На основе приведенных выше моделей создадим гибридную модель. CoClustering имеет высокую ошибку, но высокую скорость действия -> им будем отбирать топ 10 фильмов из тестовой выборки. А более точный, но затратный алгоритм KNNBaseline будем использовать для ранжирования топ 10, полученного на прошлом шаге.

In [13]:
def complex_mod(algo1, algo2, testset, uid):
    
    user_df = pd.DataFrame(testset)
    user_df.columns = ['uid', 'iid', 'rating']
    user_df = user_df[user_df.uid == uid]
    
    user_df['Сс_rating'] = user_df.apply(lambda x: algo2.predict(uid=uid, iid=x[1]).est, axis=1)
    
    user_df = user_df.sort_values(by = ['Сс_rating'], ascending=False)
    user_df = user_df[0:10]
     
    user_df['KNN_rating'] = user_df.apply(lambda x: algo1.predict(uid=uid, iid=x[1]).est, axis=1)
    
    user_df = user_df.sort_values(by = ['KNN_rating'], ascending=False)
    user_df = user_df[:10][['iid','KNN_rating']]
    
    return user_df

Лист рекомендаций для пользователя с id = 15

In [14]:
complex_mod(algo1, algo2, testset,15)

,iid,KNN_rating
10989,Star Wars: Episode VI - Return of the Jedi (1983),4.031197
9838,"Shawshank Redemption, The (1994)",4.027248
4872,Inception (2010),3.960030
14871,Fight Club (1999),3.936455
3960,Back to the Future (1985),3.887332
11981,"Matrix, The (1999)",3.886233
9733,Gladiator (2000),3.625347
14297,"Lord of the Rings: The Fellowship of the Ring,...",3.478121
7234,Ronin (1998),3.299036
8505,Mad Max: Fury Road (2015),3.244357
